In [1]:
import urllib.request as urlreq
import os
import cv2
import numpy as np
from pathlib import Path
import torch
from torch import cuda
from tqdm.notebook import tqdm
from zipfile import ZipFile
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn as fasterrcnn
import time
import shutil
import tensorflow as tf
import numpy as np
import random
import os

In [2]:

#Set seed value
seed_value = 43

os.environ['PYTHONHASHSEED'] = str(seed_value)

random.seed(seed_value)

#numpy seed
np.random.seed(seed_value)

#Tf seed
tf.random.set_seed(seed_value)

#Configure new global tensorflow session
from tensorflow.compat.v1.keras import backend as k
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads = 1
)

sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
k.set_session(sess)

In [3]:
%%capture
!pip install wandb

In [22]:
import wandb 
wandb.login()
run = wandb.init(project="fall_detection",
          config={
              'frameCount'    :50,
              'data_size'     :180,
              'dataset'       :'CMDFALL',
              'epochs'        :15,
              'seed_value'    :43,
              'split'         :0.8,
              'batch_size'    :32,
              'activation'    :'relu',
              'optimizer'     :'adam',
              'loss_fn'       :'binarycrossentropy',
              'dropout_rate'  :0.5,
              'train_split'   :0.7,
              'val_split'     :0.2,
              'test_split'    :0.1
          })
cfg = wandb.config

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [24]:
artifact = run.use_artifact('trajectory:latest', type='dataset')
artifact_dir = artifact.download()

In [26]:
artifact = run.use_artifact('boundingBox:latest', type='dataset')
artifact_dir = artifact.download()

wandb: Downloading large artifact boundingBox:latest, 402.29MB. 8423 files... Done. 0:0:0


## Logger

In [5]:
import datetime
import time
import logging
import sys

def set_logger():
  logging.basicConfig(level=logging.INFO)
  logger_t = logging.getLogger('withoutlevel')
  logger = logging.getLogger('withlevel')

  f_handler = logging.FileHandler('Falldetection.log')
  f_handler_t = logging.FileHandler('Falldetection.log')

  f_format = logging.Formatter('%(levelname)s - %(message)s')
  f_format_t = logging.Formatter('%(message)s')

  f_handler.setFormatter(f_format)
  f_handler_t.setFormatter(f_format_t)

  logger.addHandler(f_handler)
  logger_t.addHandler(f_handler_t)
  return logger, logger_t

In [6]:
import datetime
import time
t_set = lambda: datetime.datetime.now().astimezone()
t_diff = lambda t: str(t_set() - t)
w_diff = lambda t: (t_set() - t).total_seconds()
t_stamp = lambda t=None: str(t) if t else str(t_set())
t = t_set()
time.sleep(5)
t_diff(t)

'0:00:05.005281'

In [7]:
import logging
import sys

#logger without time
logging.basicConfig(level=logging.INFO)
logger_t = logging.getLogger('withoutlevel')
logger = logging.getLogger('withlevel')

f_handler = logging.FileHandler('Falldetection.log')
f_handler_t = logging.FileHandler('Falldetection.log')

f_format = logging.Formatter('%(levelname)s - %(message)s')
f_format_t = logging.Formatter('%(message)s')

f_handler.setFormatter(f_format)
f_handler_t.setFormatter(f_format_t)

logger.addHandler(f_handler)
logger_t.addHandler(f_handler_t)


In [8]:
import platform
platform.uname()
for det in platform.uname():
  logger_t.info(str(det))

INFO:withoutlevel:Linux
INFO:withoutlevel:51630f6ca72f
INFO:withoutlevel:5.4.104+
INFO:withoutlevel:#1 SMP Sat Jun 5 09:50:34 PDT 2021
INFO:withoutlevel:x86_64
INFO:withoutlevel:x86_64


In [9]:
logger_t.info('='*100)
logger_t.info("\nFalldetection project")

INFO:withoutlevel:====================================================================================================
INFO:withoutlevel:
Falldetection project


In [10]:
logger.info(f"The analysis with dataset {cfg.dataset} and frame count {cfg.frameCount} and size of the data is {cfg.data_size}.")

INFO:withlevel:The analysis with dataset CMDFALL and frame count 50 and size of the data is 180.


## Prepare dataframe

In [11]:
import pandas as pd
df_actions = pd.read_csv('http://mica.edu.vn:8000/KinectData/public/action_list.txt')
actions = dict(list(df_actions.to_records(index=False)))
df_frames = pd.read_csv('http://mica.edu.vn:8000/KinectData/public/annotation.csv')

df_video = df_frames[['setup_id','subject_id','kinect_id']]
df_video = df_video.drop_duplicates(keep='first',ignore_index=True)


In [12]:
# Cleaning the dataframe
df_frames.loc[df_frames['action_id'] == '\ufeff1', 'action_id'] = 1
df_frames.loc[df_frames['start_frame'] == '89s59','start_frame'] = 8959
# Encoding the dataframe
df_frames['start_frame'] = df_frames['start_frame'].apply(pd.to_numeric)
df_frames['action_id'] = df_frames['action_id'].apply(pd.to_numeric)
# details of the action the human performing
df_frames['action_detail'] = df_frames['action_id'].apply(lambda x: actions[x])
# details of supercategory the action resides 
df_frames['action'] = df_frames['action_id'].apply(lambda x: 'fall' if 'fall' in actions[x] else 'adl')
# Encoding the action column
df_frames['action_label'] = df_frames['action_id'].apply(lambda x: 1 if 'fall' in actions[x] else 0)

In [13]:

df_frames['frame_count'] = df_frames['stop_frame'] - df_frames['start_frame']

# Print some statistics
print(f"Total videos possible in this dataset is {len(df_frames)}")
print(f"Number of fall videos in the dataset is {len(df_frames[df_frames.action == 'fall'])}")
print(f"Number of adl videos in the dataset is {len(df_frames[df_frames.action == 'adl'])}")
print(f"Videos greater than 50 Frames are {len(df_frames[df_frames.frame_count > 50])}")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Total videos possible in this dataset is 14029
Number of fall videos in the dataset is 5472
Number of adl videos in the dataset is 8557
Videos greater than 50 Frames are 9481


In [14]:
df_frames.head()

,setup_id,subject_id,kinect_id,action_id,start_frame,stop_frame,action_detail,action,action_label,frame_count
0,1,19,2,1,0,216,walk,adl,0,216
1,1,19,2,1,286,449,walk,adl,0,163
2,1,19,2,1,584,957,walk,adl,0,373
3,1,19,2,2,960,1343,run_slowly,adl,0,383
4,1,19,2,3,1344,1746,static_jump,adl,0,402


## Download entire dataset

In [ ]:
def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

root = os.getcwd()
cmdfall = os.path.join(root,'dataset','cmdfall')


In [ ]:
values = list(df_video.iloc[0])
url = f'http://mica.edu.vn:8000/KinectData/public/colors/S{values[0]}P{values[1]}K{values[2]}.avi'
urlreq.urlretrieve(url, 'video.avi')
subframes = df_frames[(df_frames.setup_id == values[0])
                     & (df_frames.subject_id == values[1])
                     & (df_frames.kinect_id == values[2])]



In [ ]:
import time

t = time.time()
for video_idx in range(len(df_video)):
  values = list(df_video.iloc[video_idx])
  video_filename = f'S{values[0]}P{values[1]}K{values[2]}' 
  url = f'http://mica.edu.vn:8000/KinectData/public/colors/S{values[0]}P{values[1]}K{values[2]}.avi'
  video_temp_path = os.path.join(cmdfall,'temp')
  safedirs(video_temp_path)
  urlreq.urlretrieve(url, os.path.join(video_temp_path,f'{video_filename}.avi'))
  subframes = df_frames[(df_frames.setup_id == values[0])
                     & (df_frames.subject_id == values[1])
                     & (df_frames.kinect_id == values[2])]
  counter = 0
  cap = cv2.VideoCapture(os.path.join(video_temp_path, f'{video_filename}.avi'))
  frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  print(frame_count)
  while (cap.isOpened()):
    ret,frame = cap.read()
    if ret == False:
      break
    frame_details = subframes[(subframes.start_frame <= counter) & (subframes.stop_frame >= counter)].values.tolist()
    if not frame_details:
      #print(f'skipping.... {counter}')
      counter += 1
      continue
    action_detail, action, action_label = frame_details[0][-3:]
    action_dest_path = os.path.join(cmdfall, 'frame', video_filename, action, action_detail)
    safedirs(action_dest_path)
    file_path = os.path.join(action_dest_path, f'{video_filename}_{counter}_{action_label}.png')
    cv2.imwrite(file_path, frame)
    counter += 1
    if counter >= frame_count:
      cap.release()
      break
  break

print(time.time() - t)

10270.0
161.11956071853638


In [ ]:
for temp in os.listdir('/content/dataset/cmdfall/frame/S1P19K2/adl'):
  print(temp, len(os.listdir('/content/dataset/cmdfall/frame/S1P19K2/adl/'+temp)))

move_chair 487
sit_on_bed_and_stand_up 286
left_hand_pick_up 233
static_jump 403
sit_on_chair_then_stand_up 224
move_hand_and_leg 391
stagger 391
walk 755
lie_on_bed_and_sit_up 152
right_hand_pick_up 218
crawl 155
run_slowly 384


## Download videos

In [15]:
# video sequence that has greater than specified frame count is taken into account
df_frames = df_frames[df_frames.frame_count > cfg['frameCount']]
# Shuffle and retrieve the subset of Fall and Adl samples
df_fall = df_frames[df_frames.action == 'fall'].sample(frac=1,random_state=cfg['seed_value']).reset_index(drop=True)[:cfg['data_size']//2]
df_adl = df_frames[df_frames.action == 'adl'].sample(frac=1,random_state=cfg['seed_value']).reset_index(drop=True)[:cfg['data_size']//2]
df_sample = pd.concat([df_fall,df_adl]).sample(frac=1, random_state=cfg['seed_value']).reset_index(drop=True)


In [44]:
df_sample.head()

,setup_id,subject_id,kinect_id,action_id,start_frame,stop_frame,action_detail,action,action_label,frame_count
0,38,16,4,14,6412,6782,move_chair,adl,0,370
1,30,7,6,12,5509,5642,crawl,adl,0,133
2,18,6,2,19,9904,9995,lie_on_bed_and_fall_left,fall,1,91
3,3,30,3,15,6735,6790,sit_on_chair_then_fall_left,fall,1,55
4,25,39,5,5,2627,2686,left_hand_pick_up,adl,0,59


In [16]:
def safedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

logger_t.info(f'{"="*100} \n\nSTEP 1: Download dataset into the system')

root = os.getcwd()
cmdfall = os.path.join(root,'dataset','cmdfall')
subset = os.path.join(root, 'subset')

logger.info(f" dataset is downloaded in path {cmdfall}")



INFO:withoutlevel:==================================================================================================== 

STEP 1: Download dataset into the system
INFO:withlevel: dataset is downloaded in path /content/dataset/cmdfall


In [28]:
urlreq.urlretrieve('http://mica.edu.vn:8000/KinectData/public/colors/S26P28K5.avi', 'video.avi')

('video.avi', <http.client.HTTPMessage at 0x7f33f2716590>)

In [18]:
def download_dataset(df_sample, cmdfall, subset):
  # data size
  train_size = int(cfg.train_split * len(df_sample))
  validation_size = int(cfg.val_split * len(df_sample)) + train_size
  test_size = int(cfg.test_split * len(df_sample)) + validation_size
  logger.info(f'Number of files in Train set is {train_size}')
  logger.info(f'Number of files in validation set is {validation_size - train_size}')
  logger.info(f'Number of files in Test set is {test_size - validation_size}')
  # each video sequence from the data frame
  for video_idx in tqdm(range(len(df_sample))):
    t = t_set()
    # downloaded to train, validation and test in sequence
    split = 'train' if video_idx in range(train_size) else 'val' if video_idx in range(validation_size) else 'test'
    # for each record in datframe
    values = df_sample.iloc[video_idx]
    video_filename = f'S{values.setup_id}P{values.subject_id}K{values.kinect_id}' 
    url = f'http://mica.edu.vn:8000/KinectData/public/colors/{video_filename}.avi'
    # folder to store videos
    video_temp_folder = os.path.join(cmdfall,'temp')
    video_temp_file = os.path.join(video_temp_folder,f'{video_filename}.avi')
    # download video if it is not already downloaded
    if not os.path.exists(video_temp_file):
      safedirs(video_temp_folder)
      urlreq.urlretrieve(url, video_temp_file)
    # folder to store the frames of the video
    frame_folder = os.path.join(subset, split, values.action, video_filename)
    # if the frame set is already available with that video it appends the name with counter
    counter = 1
    while os.path.exists(frame_folder):
      frame_folder = os.path.join(subset,split, values.action, video_filename + f'_{counter}')
      counter += 1
    safedirs(frame_folder)
    logger_t.info(f'The file is downloaded from {url} \n extracted into path ...{frame_folder}')
    # store path of video in dataframe
    df_frames.loc[(df_frames.setup_id == values.setup_id)
                & (df_frames.subject_id == values.subject_id)
                & (df_frames.kinect_id == values.kinect_id), 'video_path'] = video_temp_file
    # read the video file
    cap = cv2.VideoCapture(video_temp_file)
    total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # randomly select the 50 frames from the video sequence
    random_start = random.randint(values.start_frame, values.stop_frame - cfg['frameCount'])
    # read specific frames from the video without reading it sequence
    for frame_idx in range(random_start, random_start + cfg['frameCount']):
      frame_file_path = os.path.join(frame_folder, f'{video_filename}_{frame_idx}_{values.action_label}.jpg')
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
      ret,frame = cap.read()
      if not ret:
        break
        raise Exception
      #print(frame_file_path)
      cv2.imwrite(frame_file_path, frame)
    logger.info(f'Elapsed time for downloading and extracting the file is {t_diff(t)}.\n')
    wandb.log({'download':{'time': w_diff(t)}})

t = t_set()
download_dataset(df_sample, cmdfall, subset)
logger.info(f'Total time for downloading and extracting entire dataset is {t_diff(t)}\n {"="*200}')
wandb.summary['Download_TotalTime'] = w_diff(t)

INFO:withlevel:Number of files in Train set is 125
INFO:withlevel:Number of files in validation set is 36
INFO:withlevel:Number of files in Test set is 18


INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S38P16K4.avi 
 extracted into path .../content/subset/train/adl/S38P16K4_1
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:01.474719.

INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S30P7K6.avi 
 extracted into path .../content/subset/train/adl/S30P7K6_1
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:01.337824.

INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S18P6K2.avi 
 extracted into path .../content/subset/train/fall/S18P6K2_1
INFO:withlevel:Elapsed time for downloading and extracting the file is 0:00:01.513891.

INFO:withoutlevel:The file is downloaded from http://mica.edu.vn:8000/KinectData/public/colors/S3P30K3.avi 
 extracted into path .../content/subset/train/fall/S3P30K3_1
INFO:withlevel:Elapsed time for downloading and extracting t

KeyboardInterrupt: ignored

## Embedding data

In [ ]:
from collections import defaultdict
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time,os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing


def person_detector(imageMatrix,label,mode,folder):
  '''
  Moving object detection by frame differencing
  input: torch tensor object of shape [batch,50,3,480,640] and labels array
  output: tuple of torch tensor [batch,1,224,224] and label object 
  '''
  info = ['adl','fall']
  bbox_path = os.path.join(os.getcwd(), 'boundingBox', mode, info[label], folder)
  logger_t.info(f'Working on creating ... {bbox_path} set')
  safedirs(bbox_path)
  temp_img = imageMatrix
  center_points = []
  # use structure similarity for detecting the motion between the frames
  for i in range(0,temp_img.shape[0] - 2):
    img1 = cv2.medianBlur(cv2.cvtColor(temp_img[i],cv2.COLOR_BGR2GRAY),5)
    img2 = cv2.medianBlur(cv2.cvtColor(temp_img[i+2],cv2.COLOR_BGR2GRAY),5)
    # score and differences in the pixel
    (score,diff) = structural_similarity(img1,img2,full=True)
    diff = (diff * 255).astype('uint8')
    # dilate the contours to avoid noise
    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    dilate_frame = cv2.dilate(thresh, None, iterations=5)
    contours = cv2.findContours(dilate_frame.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1] 
    # Calculate the contour area
    contourareas = [cv2.contourArea(c) for c in contours]
    # skip if the there is not motion detected
    if not contourareas:
      print(f'skipping {bbox_path}')
      continue
    # get the contour which has maximum area
    val = np.argmax(contourareas)
    # draw the bounding box around the contour
    x,y,w,h = cv2.boundingRect(contours[val])
    # draw the bounding box in the image
    cv2.rectangle(img1, (x, y), (x + w, y + h), (36,255,12), 2)
    path = os.path.join(bbox_path, f'image_{i}.jpg')
    # store the intermediate file
    cv2.imwrite(path, img1)
    # calculate the centers of the boxes are return it
    center = (int((x+w)/2), int((y+h)/2))
    center_points.append(center)
  return center_points

def get_transform():
  transform = tf.keras.Sequential([
                                   preprocessing.Resizing(224,224),
                                   preprocessing.Normalization()
  ])
  return transform


def image_embedding(images,label,mode,filename):
  # transforms for resizing and normalizing image
  transform = get_transform()
  info = ['adl', 'fall']
  # path to store trajectory files 
  trajectory_path = os.path.join(os.getcwd(), 'trajectories', mode, info[label])
  safedirs(trajectory_path)
  logger_t.info(f'creating trajectory image at {trajectory_path}')
  bkg = np.zeros((480,640),np.uint8)
  t=t_set()
  # retrieve the center points of the bounding rectangle
  points = person_detector(images,label,mode,filename) 
  logger.info(f'Elapsed time for Bounding box calculation is {t_diff(t)}')
  wandb.log({'boundingBox':{'time':w_diff(t)}})
  # draw line based on the points to get trajectory
  for p in range(len(points)-1):
    bkg = cv2.line(bkg,points[p],points[p+1],(255,0,0),5)
  bkg = transform(tf.expand_dims(bkg,2))
  path = os.path.join(trajectory_path, filename+'.jpg')
  # store the files in the folder
  cv2.imwrite(path, bkg.numpy())


def embedded_data(datapath):
  data_dict = defaultdict(list)
  # walkthrough all the files in the datafolder and calculate bounding box
  for root,dir,files in os.walk(datapath):
      for f in files:
        frame_path = os.path.join(root,f)
        #print(root)
        folder = os.path.basename(root)
        data_dict[folder].append(frame_path)
  # encode each image in the folder
  for key in list(data_dict.keys()):
    frames, labels = [], []
    for f in sorted(data_dict[key]):
      labels.append(int(f.split('.')[0][-1]))
      temp = f.split('/')
      mode = temp[3]
      filename = temp[5]
      img = cv2.cvtColor(cv2.imread(f),cv2.COLOR_BGR2RGB)
      frames.append(img)
    frames = np.stack(frames,axis=0)
    t=t_set()
    image_embedding(frames, np.max(labels), mode, filename)
    logger.info(f"Total time taken for {key} is {t_diff(t)}")
    wandb.log({'trajectory':{'time':w_diff(t)}})


logger_t.info(f'{"="*100} \n\nSTEP 4: Create Trajectory images\n')
t = t_set()
root = os.getcwd()
embedded_data(root + '/subset/train')
embedded_data(root + '/subset/val')
embedded_data(root + '/subset/test')
logger.info(f'Total time taken to create trajectory images: {t_diff(t)}')
wandb.summary["Trajectory_totaltime"] = w_diff(t)


INFO:withoutlevel:==================================================================================================== 

STEP 4: Create Trajectory images

INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S16P41K3 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:05.207836
INFO:withlevel:Total time taken for S16P41K3 is 0:00:05.946701
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/S42P20K3 set


skipping /content/boundingBox/train/adl/S42P20K3


INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.439637
INFO:withlevel:Total time taken for S42P20K3 is 0:00:02.475868
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/S41P29K6_1 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.419611
INFO:withlevel:Total time taken for S41P29K6_1 is 0:00:02.453037
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/S55P18K3 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.416165
INFO:withlevel:Total time taken for S55P18K3 is 0:00:02.449627
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/S13P42K4 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.433649
INFO:withlev

skipping /content/boundingBox/train/fall/S25P39K6
skipping /content/boundingBox/train/fall/S25P39K6
skipping /content/boundingBox/train/fall/S25P39K6
skipping /content/boundingBox/train/fall/S25P39K6


INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:04.974073
INFO:withlevel:Total time taken for S25P39K6 is 0:00:05.006308
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/fall/S34P27K6 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:04.924932
INFO:withlevel:Total time taken for S34P27K6 is 0:00:04.956620
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/S34P27K7 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.448547
INFO:withlevel:Total time taken for S34P27K7 is 0:00:02.486476
INFO:withoutlevel:creating trajectory image at /content/trajectories/train/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/train/adl/S24P37K1 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.462170
INFO:withlevel

...

INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.410642
INFO:withlevel:Total time taken for S27P4K5 is 0:00:02.450203
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/fall/S28P9K3 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.432955
INFO:withlevel:Total time taken for S28P9K3 is 0:00:02.466370
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/fall/S19P3K1 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.427030
INFO:withlevel:Total time taken for S19P3K1 is 0:00:02.460376
INFO:withoutlevel:creating trajectory image at /content/trajectories/val/fall
INFO:withoutlevel:Working on creating ... /content/boundingBox/val/fall/S53P26K3 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.459010
INFO:withlevel:Total time t

skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6


INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.442296


skipping /content/boundingBox/test/adl/S44P5K6
skipping /content/boundingBox/test/adl/S44P5K6


INFO:withlevel:Total time taken for S44P5K6 is 0:00:02.480831
INFO:withoutlevel:creating trajectory image at /content/trajectories/test/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/test/adl/S24P37K2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.426671
INFO:withlevel:Total time taken for S24P37K2 is 0:00:03.090286
INFO:withoutlevel:creating trajectory image at /content/trajectories/test/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/test/adl/S10P12K2 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.414610
INFO:withlevel:Total time taken for S10P12K2 is 0:00:02.447329
INFO:withoutlevel:creating trajectory image at /content/trajectories/test/adl
INFO:withoutlevel:Working on creating ... /content/boundingBox/test/adl/S36P34K7 set
INFO:withlevel:Elapsed time for Bounding box calculation is 0:00:02.441432
INFO:withlevel:Total time taken for S36P34K7 is 0:00:02.478124
INFO:withoutlevel:creating trajecto

In [31]:
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Data generators\n')
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
logger.info(f'batch size: {cfg.batch_size}')
root = os.getcwd()
train = datagen.flow_from_directory(root + '/artifacts/trajectory:v7/train',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)
valid = datagen.flow_from_directory(root + '/artifacts/trajectory:v7/val',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)
test = datagen.flow_from_directory(root + '/artifacts/trajectory:v7/test',target_size=(224, 224),color_mode='grayscale',batch_size=cfg.batch_size)

INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Data generators

INFO:withlevel:batch size: 32


Found 117 images belonging to 2 classes.
Found 33 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


## training pipeline


In [42]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,Dense,Dropout
from tensorflow.keras.metrics import Precision, Recall
logger_t.info(f'{"="*100} \n\nSTEP 5: Create Model and train the model\n')



model = tf.keras.Sequential([
              tf.keras.Input(shape=(224,224,1)),
              Conv2D(32,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              Conv2D(64,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              Conv2D(128,3,strides=2,padding='same',activation='relu',use_bias=False),
              MaxPool2D(),
              GlobalAveragePooling2D(),
              Dense(128),
              Dropout(0.5),
              Dense(64),
              Dropout(0.5),
              Dense(2,activation='sigmoid')])


model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', Precision(), Recall()])
logger.info(f'{model.summary()}')



INFO:withoutlevel:==================================================================================================== 

STEP 5: Create Model and train the model



INFO:withlevel:None


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 112, 112, 32)      288       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        18432     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)         73728     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 3, 3, 128)         0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 128)              

In [33]:
best_model = wandb.restore('model-best.h5', run_path='manikandan/fall_detection/39zrjl4w')
best_model.name

'/content/wandb/run-20210630_113356-2fi8iucz/files/model-best.h5'

In [34]:
best_model = tf.keras.models.load_model(best_model.name)
best_model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-2), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', Precision(), Recall()])

In [38]:
print('test')
best_model.evaluate(test)
print('validation')
best_model.evaluate(valid)
print('train')
best_model.evaluate(train)


test
1/1 [==============================] - 0s 96ms/step - loss: 6.8388 - accuracy: 0.5000 - precision_1: 0.5294 - recall_1: 0.5000
validation
2/2 [==============================] - 0s 13ms/step - loss: 5.9628 - accuracy: 0.5152 - precision_1: 0.5312 - recall_1: 0.5152
train
4/4 [==============================] - 0s 90ms/step - loss: 7.0739 - accuracy: 0.4872 - precision_1: 0.4828 - recall_1: 0.4786


[7.073879241943359,
 0.4871794879436493,
 0.48275861144065857,
 0.47863247990608215]

In [43]:
from wandb.keras import WandbCallback
history = best_model.fit_generator(generator=train,
                              validation_data=valid,
                              epochs=cfg.epochs,
                              verbose=1,
                              callbacks=[WandbCallback()])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
4/4 [==============================] - 3s 545ms/step - loss: 2.9752 - accuracy: 0.4872 - precision_3: 0.5042 - recall_3: 0.5128 - val_loss: 1.4951 - val_accuracy: 0.5152 - val_precision_3: 0.4231 - val_recall_3: 0.3333
Epoch 2/15
4/4 [==============================] - 1s 378ms/step - loss: 2.2808 - accuracy: 0.4872 - precision_3: 0.5122 - recall_3: 0.3590 - val_loss: 0.8246 - val_accuracy: 0.5152 - val_precision_3: 0.5250 - val_recall_3: 0.6364
Epoch 3/15
4/4 [==============================] - 1s 371ms/step - loss: 1.4367 - accuracy: 0.5214 - precision_3: 0.5033 - recall_3: 0.6496 - val_loss: 0.9912 - val_accuracy: 0.4848 - val_precision_3: 0.5091 - val_recall_3: 0.8485
Epoch 4/15
4/4 [==============================] - 1s 372ms/step - loss: 0.9603 - accuracy: 0.5726 - precision_3: 0.5468 - recall_3: 0.6496 - val_loss: 0.7433 - val_accuracy: 0.6061 - val_precision_3: 0.5152 - val_recall_3: 0.5152
Epoch 5/15
4/4 [==============================] - 1s 373ms/step - loss: 0.7631 -

In [ ]:
results = model.evaluate(test)
wandb.summary['test_loss'] = results[0]
wandb.summary['test_accuracy'] = results[1]
wandb.summary['test_precision'] = results[2]
wandb.summary['test_recall'] = results[3]

1/1 [==============================] - 0s 153ms/step - loss: 3.2981 - accuracy: 0.5556 - precision: 0.0000e+00 - recall: 0.0000e+00


## Artifact

In [ ]:
bbox = wandb.Artifact('boundingBox', type='dataset')
bbox.add_dir('boundingBox')
wandb.log_artifact(bbox)

wandb: Adding directory to artifact (./boundingBox)... Done. 20.7s


In [ ]:
trajectory = wandb.Artifact('trajectory', type='dataset')
trajectory.add_dir('trajectories')
wandb.log_artifact(trajectory)

wandb: Adding directory to artifact (./trajectories)... Done. 0.5s


In [ ]:
 wandb.finish()

_runtime,10777
_timestamp,1624606536
_step,530
Download_TotalTime,8662.61427
Trajectory_totaltime,498.22305
epoch,14
loss,0.23474
accuracy,0.95726
precision_1,0.94828
recall_1,0.94017
val_loss,0.90937


_runtime,▁▁▂▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
_timestamp,▁▁▂▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁
accuracy,▁▁▂▂▄▄▃▆▆▆▇▇███
precision_1,▁▁▂▃▄▃▃▅▅▅▆▇▇██
recall_1,▁▂▂▂▂▃▃▅▇▆▇▇▇██
val_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,██▃▅▃▃▅▁▅▆▃▅▅▁▆
val_precision_1,█▅▁▃▂▅▆▄▄▄▂▃▄▂▆
